## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras import backend as K
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
import os

Using TensorFlow backend.


In [6]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
# 資料路徑
DATASET_PATH  = 'sample'

# 影像大小
IMAGE_SIZE = (32, 32)

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 20

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow(x_train, y_train,
                                   #target_size=IMAGE_SIZE,
                                   #interpolation='bicubic',
                                   #class_mode='categorical',
                                   shuffle=True,
                                   batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow(x_test, y_test,
                                   #target_size=IMAGE_SIZE,
                                   #interpolation='bicubic',
                                   #class_mode='categorical',
                                   shuffle=False,
                                   batch_size=BATCH_SIZE)

# 輸出各類別的索引值
#for cls, idx in train_batches.class_indices.items():
#    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

import math
# 訓練模型
net_final.fit_generator(train_batches,
                        #steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        steps_per_epoch = math.ceil(len(x_train)) / BATCH_SIZE,
                        validation_data = valid_batches,
                        #validation_steps = valid_batches.samples // BATCH_SIZE,
                        validation_steps = math.ceil(len(x_test)) // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

C:\Users\Hsin-PC\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
__________________________________________________________________________________________________
None
Epoch 1/20
6250/6250 [==============================] - 9003s 1s/step - loss: 2.9162 - acc: 0.1897 - val_loss: 2.1108 - val_acc: 0.3293
Epoch 2/20
6250/6250 [==============================] - 8974s 1s/step - loss: 2.0872 - acc: 0.2893 - val_loss: 1.6840 - val_acc: 0.4611
Epoch 3/20
6250/6250 [==============================] - 9021s 1s/step - loss: 1.7675 - acc: 0.3763 - val_loss: 1.6579 - val_acc: 0.5279
Epoch 4/20
6250/6250 [==============================] - 9076s 1s/step - loss: 1.6366 - acc: 0.4257 - val_loss: 1.4213 - val_acc: 0.5681
Epoch 5/20
6250/6250 [==============================] - 9090s 1s/step - loss: 1.5326 - acc: 0.4662 - val_loss: 1.2235 - val_acc: 0.6154
Epoch 6/20
6250/6250 [==============================] - 9080s 1s/step - loss: 1.4710 - acc: 0.4876 - val_loss: 1.1635 - val_acc: 0.63

In [16]:
from keras.models import load_model
from keras.preprocessing import image
import sys
import numpy as np

# 從參數讀取圖檔路徑
files = sys.argv[1:]

# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

cls_list = ['cats', 'dogs']

# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))

FileNotFoundError: [Errno 2] No such file or directory: '-f'